In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

import copy
import traceback
import datetime
import joblib
import re
import os
import random
import string

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch import nn
from torch.nn import functional as F

from tqdm.notebook import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import pymorphy2
from sklearn.base import BaseEstimator, TransformerMixin

In [3]:
class Config:
    seed = 42
    positive_file = "../data/positive.csv"
    negative_file = "../data/negative.csv"
    russian_stop_words = "../data/russian_stop_words.txt"
    english_stop_words = "../data/english_stop_words.txt"
    test_size = 0.3
    
config = Config()

In [4]:
def init_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic=True
    os.environ['PYTHONHASHSEED'] = str(seed)
    
init_random_seed(config.seed)

In [16]:
train = pd.read_csv("../data/train_processed_data.csv", index_col=False)
validate = pd.read_csv("../data/validate_processed_data.csv", index_col=False)
test = pd.read_csv("../data/test_processed_data.csv", index_col=False)

In [22]:
X_train, X_valid, X_test = train['text'].values.tolist(), validate['text'].values.tolist(), test['text'].values.tolist()
y_train, y_valid, y_test = train['target'].values.tolist(), validate['target'].values.tolist(), test['target'].values.tolist(),

In [23]:
X_train[:10]

['оставаться самый нужный и самый близкие ) весь остальной уходить ) и я только рада ) потому что я никогда сам не понять нужный я человек или нет )',
 'такой приятный чувство , когда ты знаешь , что подарить человек и ты на уверить , что он быть рад ! : ) ) теперь ждать новый год ! : )',
 'день начинаться с лень вообще ничто делать не хотеть . даже рука шевелить ( ( ( ничто пройти . . .',
 'at_user at_user ксюша поход вплотную там суп заняться )',
 'at_user с днём рождение at_user , творческий успех ты ! )',
 '1 вопрос , ответ . ничто лишний . защищать курсач секунда ) )',
 'весь , пора отказаться от кофе , а то иначе мой ближний будущее это жёлтый зуб ( (',
 'at_user at_user я снова чувствовать как рушиться мой гениальный план . . ( ( ( рахманинааа , виза лишний нет ? : в я срочно ! ! !',
 'at_user а он не говорить кто он быть спрашивать ну или что ? просто как бы два группа сразу ( (',
 'неделя ад начаться ; ( ( как же не хотеться вставать ( (']

In [24]:
y_train[:10]

[1, 1, 0, 1, 1, 1, 0, 0, 0, 0]

In [25]:
pipeline = Pipeline([
    ('bow', CountVectorizer(ngram_range=(1, 1), min_df=5)),  # strings to token integer counts
    ('tfidf', TfidfTransformer(norm="l2", smooth_idf=True, use_idf=True)),  # integer counts to weighted TF-IDF scores
    ('classifier', RandomForestClassifier()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

parameters = {
        "bow__min_df": [1, 2, 3, 4, 5],
        "bow__ngram_range": [(1, 1), (1, 2)],
        "tfidf__use_idf": (True, False),
        "tfidf__smooth_idf": (True, False),
        "classifier": [LogisticRegression()],
        "classifier__penalty": ['l2','l1'],
        "classifier__C": np.logspace(0, 4, 10)
}

In [26]:
# grid = GridSearchCV(pipeline, cv=5, param_grid=parameters, verbose=1, n_jobs=20, scoring=["accuracy", "precision", "recall", "f1", "roc_auc"], refit="f1")
grid = GridSearchCV(pipeline, cv=5, param_grid=parameters, verbose=1, n_jobs=20, scoring="f1")
grid.fit(X_train,y_train)
print(f"Best Model: {grid.best_score_} using {grid.best_params_}")

Fitting 5 folds for each of 800 candidates, totalling 4000 fits


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   10.6s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   52.8s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:  3.2min
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed: 13.0min
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed: 16.2min
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed: 20.4min
[Parallel(n_jobs=20)]: Done 2410 tasks      | elapsed: 24.7min
[Parallel(n_jobs=20)]: Done 3160 tasks      | elapsed: 29.1min
[Parallel(n_jobs=20)]: Done 4000 out of 4000 | elapsed: 33.8min finished


Best Model: 0.7724441397251937 using {'bow__min_df': 1, 'bow__ngram_range': (1, 2), 'classifier': LogisticRegression(C=2.7825594022071245), 'classifier__C': 2.7825594022071245, 'classifier__penalty': 'l2', 'tfidf__smooth_idf': True, 'tfidf__use_idf': True}


In [27]:
print(f"Best Model: {grid.best_score_} using {grid.best_params_}")

Best Model: 0.7724441397251937 using {'bow__min_df': 1, 'bow__ngram_range': (1, 2), 'classifier': LogisticRegression(C=2.7825594022071245), 'classifier__C': 2.7825594022071245, 'classifier__penalty': 'l2', 'tfidf__smooth_idf': True, 'tfidf__use_idf': True}


In [32]:
# save best model to current working directory
joblib.dump(grid, "models/02_sklearn_pipeline_linear_classifier.pkl")
# load from file and predict using the best configs found in the CV step
model = joblib.load("models/02_sklearn_pipeline_linear_classifier.pkl" )
# get predictions from best model above
y_preds = model.predict(X_test)
print('accuracy score: ', accuracy_score(y_test, y_preds))
print('\n')
print('confusion matrix: \n', confusion_matrix(y_test,y_preds))
print('\n')
print(classification_report(y_test, y_preds))

accuracy score:  0.766702669341151


confusion matrix: 
 [[16569  5920]
 [ 4664 18214]]


              precision    recall  f1-score   support

           0       0.78      0.74      0.76     22489
           1       0.75      0.80      0.77     22878

    accuracy                           0.77     45367
   macro avg       0.77      0.77      0.77     45367
weighted avg       0.77      0.77      0.77     45367



In [33]:
y_preds = model.predict(X_valid)
f1_score(y_valid, y_preds, average="macro")

0.7650356758817975